In [1]:
import googlemaps as gm
from datetime import datetime
import folium
from IPython.display import display
import re

In [2]:
# Set up Google Maps client using personal API key
gmap = gm.Client(key='Your API Key')

In [3]:
# Enter the list of apartments to visit as well as a starting location

apartments = [
    "30 W 13th St",
    "303 Mercer St",
    "234 E 23rd St",
    "225 E 19th St"
]

starting_location =  "718 Broadway"

In [4]:
# Create a function that retrieves directions (walking and driving only) using the Google Maps API
# This function also returns the duration for each "leg" of the journey 

def get_directions(origin, destination):
    driving_directions = gmap.directions(origin, destination, mode='driving')
    walking_directions = gmap.directions(origin, destination, mode='walking')
    
    driving_duration = driving_directions[0]['legs'][0]['duration']['value']
    walking_duration = walking_directions[0]['legs'][0]['duration']['value']
    
    return driving_directions[0]['legs'][0], walking_directions[0]['legs'][0], driving_duration, walking_duration

In [5]:
# Create a function that returns the latitude and longitude for an address

def get_coordinates(address):
    geocode_result = gmap.geocode(address)
    if geocode_result:
        location = geocode_result[0]['geometry']['location']
        latitude = location['lat']
        longitude = location['lng']
        return latitude, longitude
    else:
        return None

In [27]:
# Create a function that will calculate the apartment tour route using the k nearest neighbors algorithm
# In this case, k is implicitly set to the length of the apartments list

def calculate_route(apartments, starting_location):
    remaining_apartments = apartments.copy()
    current_location = starting_location 
    total_duration = 0 
    route = [current_location] 

    while remaining_apartments: # This loop will execute while there are remaining apartments in the copied list
        nearest_apartment = None
        min_duration = float('inf') # Use inf to initialize min_duration so that any apartment selected will start the algorithm
        transit_steps = []

        for apartment in remaining_apartments:
            driving_leg, walking_leg, driving_duration, walking_duration = get_directions(current_location, apartment)

            # Check if walking or driving is faster 
            if driving_duration < walking_duration: 
                duration = driving_duration
                directions = driving_leg
                mode = 'DRIVING'
            else:
                duration = walking_duration
                directions = walking_leg
                mode = 'WALKING'
            
            # Selects an apartment with duration less than inf to start and then keeps looking for lower duration apartments
            if duration < min_duration: 
                min_duration = duration
                nearest_apartment = apartment
                transit_steps = [
                    step for step in directions['steps'] if step['travel_mode'] == mode
                ]
        
        # Update while loop variables, adds selected apartment route to the route list, and removes the apartment from remaining_apartments
        total_duration += min_duration 
        current_location = nearest_apartment 
        route.append(current_location) 
        remaining_apartments.remove(nearest_apartment)

       # Print directions
        print(f"\nDirections to {current_location}:")
        for step in transit_steps:
            instruction = re.sub(r'<.*?>', '', step['html_instructions'])
            if mode == 'DRIVING':
                print(f"- Drive: {instruction}")
            else:
                print(f"- Walk: {instruction}")
    
    # Returns coordinates for map visualization purposes
    coordinates = [(get_coordinates(location)[0], get_coordinates(location)[1]) for location in route]
    return route, total_duration, coordinates

In [33]:
# Create a function that visualizes the apartment tour on a map

def display_route(route, total_duration, coordinates):
    print(f"\nOptimal Route:")
    for i, apartment in enumerate(route):
        print(f"{i+1}. {apartment}")
    print(f"Total Travel Time: {total_duration/60} minutes")

    # Create a folium map centered around NYC
    map = folium.Map(location=coordinates[0], zoom_start=12, tiles='CartoDB Positron')

    # Plot the coordinates as markers on the map ending with a red marker for the final stop
    for i, (lat, lng) in enumerate(coordinates):
        tooltip = f"Location {i+1}: {route[i]}"
        if i == len(coordinates) - 1:
            tooltip += " (Final Stop)"
        icon_color = 'red' if i == len(coordinates) - 1 else 'blue'
        folium.Marker([lat, lng], tooltip=tooltip, icon=folium.Icon(color=icon_color)).add_to(map)

    # Add a green marker for the starting location
    start_lat, start_lng = coordinates[0]
    tooltip_start = f"Starting Location: {route[0]}"
    folium.Marker([start_lat, start_lng], icon=folium.Icon(color='green'), tooltip=tooltip_start).add_to(map)

    # Display the map
    map.save("route_map.html")  # Save the map as an HTML file
    display(map)  # Display the map in Jupyter

In [34]:
# Use the calculate_route function and store the returned outputs as variables 
route, total_distance, coordinates = calculate_route(apartments, starting_location)

# Use the display_route function with the calculate_route output variables as the arguments
display_route(route, total_distance, coordinates)


Directions to 303 Mercer St:
- Drive: Head northeast on Broadway toward Waverly Pl
- Drive: Turn left onto Waverly Pl
- Drive: Turn right onto Mercer StDestination will be on the right

Directions to 30 W 13th St:
- Drive: Head southwest on Mercer St toward Waverly Pl
- Drive: Turn right at the 2nd cross street onto Washington Pl
- Drive: Turn right onto Washington Square E
- Drive: Continue straight onto University Pl
- Drive: Turn left onto E 13th StDestination will be on the left

Directions to 234 E 23rd St:
- Drive: Head northwest on W 13th St toward 6th Ave/Ave of the Americas
- Drive: Turn right at the 1st cross street onto 6th Ave/Ave of the AmericasPass by GNC (on the left)
- Drive: Turn right onto W 23rd StPass by Citizens (on the left in 0.6 mi)Destination will be on the right

Directions to 225 E 19th St:
- Drive: Head southeast on E 23rd St toward 2nd Ave
- Drive: Turn right at the 1st cross street onto 2nd Ave
- Drive: Turn right onto E 19th StDestination will be on the 